### Récupération des données de statistiques d'Aftercinema

#### Vérification du bon fonctionnement de l'environnement + installations + imports

In [ ]:
print("hello")

In [ ]:
%pip install requests
%pip install pandas
%pip install sqlalchemy
%pip install psycopg2

In [ ]:
import os
import json
import requests
import pandas as pd
from sqlalchemy import create_engine, Table, Column, String, MetaData, Date, JSON
from datetime import datetime

#### 1.Récupération des données PostHog

In [ ]:
token = os.getenv("POSTHOG_API_KEY")

headers = {"Authorization": f"Bearer {token}"}

with open("./POSTHOG_QUERIES.json", 'r') as fichier:
    queries = json.load(fichier)

events_data = {}

for event in ["Page viewed","Platform button clicked"]:
    data = {
        "query": {
            "kind": "HogQLQuery",
            "query": queries[event]
        }
    }

    request = requests.post("https://eu.posthog.com/api/projects/20861/query",headers=headers,json=data)

    data = request.json()

    df = pd.DataFrame(data["results"])
    df.columns = data["columns"]

    events_data[event] = df

Traitement des données pour le graphique PostHog - plateformes

In [ ]:
df_platforms = events_data["Platform button clicked"]
df_platforms = df_platforms.groupby('platform').size().reset_index(name='count')
result_platforms = df_platforms.to_dict(orient='records')

Traitement des données pour le graphique PostHog - pages

In [ ]:
df_pages = events_data["Page viewed"]
df_pages['time'] = pd.to_datetime(df_pages['time'])
df_pages['year_month'] = df_pages['time'].dt.tz_localize(None).dt.to_period('M')
month_map = {1: 'jan', 2: 'fév', 3: 'mar', 4: 'avr', 5: 'mai', 6: 'jun',7: 'jui', 8: 'aoû', 9: 'sep', 10: 'oct', 11: 'nov', 12: 'déc'}
df_pages['month'] = df_pages['year_month'].dt.month.map(month_map) + " " + (df_pages['year_month'].dt.year % 100).astype(str)
df_pages['page'] = df_pages['url'].apply(lambda x: '/listen' if '/listen' in x else '/')
df_pages = df_pages.groupby(['month', 'page']).size().reset_index(name='count')
df_pages = df_pages.pivot(index='month', columns='page', values='count').fillna(0).reset_index()
result_pages = df_pages.to_dict(orient='records')

#### Stockage du résultat

In [ ]:
data_to_insert = [
    {
        "data_name":"PostHog - Page viewed",
        "data":result_pages,
        "date": datetime.now().strftime("%Y-%m-%d %H:%M")
    },
    {
        "data_name":"PostHog - Platform button clicked",
        "data":result_platforms,
        "date": datetime.now().strftime("%Y-%m-%d %H:%M")
    }
]

engine = create_engine(os.getenv("POSTGRESQL_CONN_STRING"))

metadata = MetaData()
table = Table('stats_data', metadata,
              Column('data_name', String),
              Column('data', JSON),
              Column('date', String))

metadata.create_all(engine)

for line in data_to_insert:
    with engine.connect() as connection:
        with connection.begin() as transaction:
            connection.execute(table.insert().values(data_name=line["data_name"],data=line["data"],date=line["date"]))
            transaction.commit()